In [1]:
import pickle
import utils
import json

In [ ]:
aux_path='auxiliary_data'
all_docs_file = '%s/docs.conll' % aux_path # 
more_docs_file='%s/new.conll'  % aux_path #
relevant_docs_file='%s/final_docs.p'  % aux_path # 

input_path='../data/input/partial'
out_pickle = '%s/text/docs.p' % input_path

### 1) Get and store texts to disk

In [ ]:
# Load all necessary components
all_docs_data=utils.load_conll_data(all_docs_file)

more_docs_data=utils.load_conll_data(more_docs_file)

with open(relevant_docs_file, 'rb') as f:
    relevant_incdocs=pickle.load(f)
    


In [ ]:
# Merge initial and added documents
for more_doc, more_doc_data in more_docs_data.items():
    if more_doc not in all_docs_data.keys():
        all_docs_data[more_doc]=more_doc_data

In [ ]:
# Combine document IDs from all incidents
relevant_docs=set()
for inc, docs in relevant_incdocs.items():
    relevant_docs |= set(docs)
    
print(len(relevant_docs))

In [ ]:
# Keep only the data for the relevant documents
docs_data={}
for k,v in all_docs_data.items():
    if k in relevant_docs:
        docs_data[k]=all_docs_data[k]
        
len(docs_data)

In [ ]:
docs_data['39890c5f9109dcf8e892c4fc6eaa1fd6']

In [ ]:
with open(out_pickle, 'wb') as p:
    pickle.dump(docs_data, p)

### 2) Increase ambiguity by giving everyone the same first or last name

In [ ]:
input_path='../data/input/partial'
gva_input_annotations_file = '%s/annotation/participants_input.p' % input_path



#### 2a) Increase ambiguity by giving everyone the same first name

In [ ]:
new_first_name = "John"

gva_samefirstname_annotations = '%s/annotation/participants_samefirstname.p' % input_path

with open(gva_input_annotations_file, 'rb') as f:
    gva_input_annotations=pickle.load(f)
    for doc_id, data in gva_input_annotations.items():
        for participant_id, participant_info in data.items():
            if 'Name' in participant_info:
                old_name=participant_info['Name']
                first_name, *other_names = old_name.split()
                participant_info['Name']=' '.join([new_first_name] + other_names)
                print(old_name, participant_info["Name"])
    with open(gva_samefirstname_annotations, 'wb') as w:
        pickle.dump(gva_input_annotations, w)

#### 2b) Increase ambiguity by giving everyone the same last name

In [ ]:
new_last_name = "Smith"

gva_samelastname_annotations = '%s/annotation/participants_samelastname.p' % input_path

with open(gva_input_annotations_file, 'rb') as f:
    gva_input_annotations=pickle.load(f)
    for doc_id, data in gva_input_annotations.items():
        for participant_id, participant_info in data.items():
            if 'Name' in participant_info:
                old_name=participant_info['Name']
                *other_names, last_name = old_name.split()
                participant_info['Name']=' '.join(other_names + [new_last_name])
                print(old_name, participant_info["Name"])
    with open(gva_samelastname_annotations, 'wb') as w:
        pickle.dump(gva_input_annotations, w)

### 3. Transform gold data to JSON with cluster IDs

In [ ]:
which_partition='partial'
gold_file = '../data/gold/%s/participants_gold.p' % which_partition
out_gold_file = '../data/gold/%s/participants.json' % which_partition

with open(gold_file, 'rb') as g:
    gold_data = pickle.load(g)

In [ ]:
gold_json=utils.transform_gold_to_json(gold_data)

In [ ]:
with open(out_gold_file, 'w') as w:
    json.dump(gold_json, w)